Εισαγωγή του αρχικού csv 

In [1]:
import sys
sys.path.append('..')

import importcsv
import wordtools

print('Importing csv file...........')
dataset = importcsv.opencsv('../Greek_Parliament_Proceedings_1989_2020.csv')  


Importing csv file...........


Ομιλίες με χαρακτήρες λιγότερους από 150 απαλείφονται.

In [2]:
print('Remove Small Speeches...........')
dataset = dataset.drop(wordtools.removesmallspeechesindex(dataset, 200)).reset_index(drop=True)

Remove Small Speeches...........


Αφαιρούνται οι ομιλίες του προεδρείου της βουλής γιατί δεν περιέχουν πολιτικό περιεχόμενο, οι ομιλίες τους αφορούν μόνο τη διαδικασία του κοινοβουλίου.

In [3]:
stringto = 'προεδρος βουλης'
mask = dataset['roles'].str.contains(stringto, na=False)
dataset = dataset[~mask]

Στο επόμενο στάδιο για όλες τις ομιλίες βγαίνουν οι τόνοι, γίνονται μικρά τα γράμματα, βγαίνουν λέξεις από το αρχείο stopwords1 και γίνεται stemming με το greek-stemmer

In [4]:
dataset['processed_speeches'] = dataset['speech'].apply(wordtools.process_string)

Removing the rows that have at least one empty cell

In [5]:
dataset = dataset.dropna().reset_index(drop=True)

Changing the names of the political parties in order to be easier to be found

In [6]:
dataset.loc[:,('processed_speeches')] = wordtools.change_party_names(dataset)

remove numbers except 1453 and numbers between 1800-2039 (years)

In [7]:
import re

def remove_numbers(text):
    pattern = re.compile(r'\b(?!(1453 | 18[0-9]{2} | 19[0-9]{2} | 20[0-3][0-9])\b)\d+\b')
    text = re.sub(pattern, '', text)    
    return text

dataset.loc[:,('processed_speeches')] = dataset['processed_speeches'].apply(remove_numbers)

Create a new column with the year extracted from the sitting_date

In [8]:
# function to extract the year from a date
from datetime import datetime
def extr_year(date):
    dateobjext = datetime.strptime(date, '%d/%m/%Y')
    return dateobjext.year

#Apply the function to column 'sitting date' and store it in a new column 'year'
dataset['year'] = dataset['sitting_date'].apply(extr_year)

In [9]:
df = dataset['speech']

# Calculate the length of each row in bytes
row_lengths_bytes = df.apply(lambda row: sys.getsizeof(row))

# Define the threshold (128 KB in bytes)
threshold_bytes = 128 * 1024

# Use a boolean mask to filter rows based on the condition
filtered_rows = df[row_lengths_bytes > threshold_bytes]

# remove large speeches that can not be stored in the App (TextField max size 131702)
small_dataset = dataset.drop(index=filtered_rows.index.to_list()).reset_index(drop=True)

Save the data to a new file

In [10]:
importcsv.writetodisk(dataset, '../processedspeechfinal.csv')